# Segmenting and Clustering in Toronto


## Scrape the data
Download the dependencies

In [1]:
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analysis
import requests
!pip install folium # map rendering library
import folium # map rendering library
!pip install geocoder
import geocoder

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors
import matplotlib.pyplot as plt

# import k-means from clustering stage
from sklearn.cluster import KMeans

print('Libraries imported.')

Libraries imported.


### Question 1
The code below scrapes Wikipedia for postcodes in Toronto beginning with the letter M.

In [2]:
WIKI_URL = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

from bs4 import BeautifulSoup

req = requests.get(WIKI_URL)
soup = BeautifulSoup(req.content, 'lxml')
postcode_table = soup.find('table', class_='wikitable sortable')

prev_pcode = '' # Used to compare the current PostalCode with the previous for grouping Neighborhoods

Postcode = []
Borough = []
Neighborhood = []

for row in postcode_table.findAll('tr'):
    cells=row.findAll('td')
    if len(cells) == 3:
        # Gather Postcode, borough and neighborhood data
        curr_pcode = cells[0].find(text=True).strip()
        curr_borough = cells[1].find(text=True).strip()
        curr_nhood = cells[2].find(text=True).strip()
        
        # If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.
        if curr_nhood == 'Not assigned':
            curr_nhood = curr_borough
        
        # Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.
        if curr_borough != 'Not assigned':
            # Do we have a new postcode?
            if prev_pcode != curr_pcode:
                Postcode.append(curr_pcode)
                Borough.append(curr_borough)
                Neighborhood.append(curr_nhood)
                
                # Update the previous postcode
                prev_pcode = curr_pcode
            else:
                # Postcode is the same as previous then append the neighborhood to the existing one
                last_rec = len(Neighborhood) - 1
                Neighborhood[last_rec] = Neighborhood[last_rec] + ', ' + curr_nhood

toronto_data=pd.DataFrame(Postcode,columns = ['PostalCode'])
toronto_data['Borough'] = Borough
toronto_data['Neighborhood'] = Neighborhood
toronto_data

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront, Regent Park"
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
4,M7A,Queen's Park,Queen's Park
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Rouge, Malvern"
7,M3B,North York,Don Mills North
8,M4B,East York,"Woodbine Gardens, Parkview Hill"
9,M5B,Downtown Toronto,"Ryerson, Garden District"


In [3]:
print(toronto_data.shape)

(103, 3)


## Question 2
The following code uses the Geocoder library to obtain latitude/longitude information for each postcode obtained in Q1

In [4]:
Latitude = []
Longitude = []

for pcode in Postcode:
    # Gather latitude/longitude data using the Geocoder API
    g = geocoder.arcgis('{}, Toronto, Ontario'.format(pcode))
    Latitude.append(g.latlng[0])
    Longitude.append(g.latlng[1])

toronto_data['Latitude'] = Latitude
toronto_data['Longitude'] = Longitude
toronto_data

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.752440,-79.329271
1,M4A,North York,Victoria Village,43.730421,-79.313320
2,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.655120,-79.362640
3,M6A,North York,"Lawrence Heights, Lawrence Manor",43.723125,-79.451589
4,M7A,Queen's Park,Queen's Park,43.661102,-79.391035
5,M9A,Etobicoke,Islington Avenue,43.662242,-79.528379
6,M1B,Scarborough,"Rouge, Malvern",43.811525,-79.195517
7,M3B,North York,Don Mills North,43.749195,-79.361905
8,M4B,East York,"Woodbine Gardens, Parkview Hill",43.707535,-79.311773
9,M5B,Downtown Toronto,"Ryerson, Garden District",43.657363,-79.378180


In [5]:
# Obtain latitude/longitude of Toronto
g = geocoder.arcgis('Toronto, Ontario')
tor_lat = g.latlng[0]
tor_long = g.latlng[1]

# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[tor_lat, tor_long], zoom_start=10)

# add markers to map
for lat, lng, pcode in zip(toronto_data['Latitude'], toronto_data['Longitude'], toronto_data['PostalCode']):
    label = folium.Popup(pcode, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  

map_toronto

## Question 3
The following code uses the Foursquare API to obtain nearby venue information for each neighborhood

In [6]:
# The code was removed by Watson Studio for sharing.

Foursquare credentails loaded


In [7]:
# A function that returns nearby venues for a given latitude/longitude
def getNearbyVenues(names, latitudes, longitudes, radius=500, limit=100):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            limit)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['PostalCode', 
                  'PostalCode Latitude', 
                  'PostalCode Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [8]:
toronto_venues = getNearbyVenues(names=toronto_data['PostalCode'],
                                   latitudes=toronto_data['Latitude'],
                                   longitudes=toronto_data['Longitude']
                                  )

M3A
M4A
M5A
M6A
M7A
M9A
M1B
M3B
M4B
M5B
M6B
M9B
M1C
M3C
M4C
M5C
M6C
M9C
M1E
M4E
M5E
M6E
M1G
M4G
M5G
M6G
M1H
M2H
M3H
M4H
M5H
M6H
M1J
M2J
M3J
M4J
M5J
M6J
M1K
M2K
M3K
M4K
M5K
M6K
M1L
M2L
M3L
M4L
M5L
M6L
M9L
M1M
M2M
M3M
M4M
M5M
M6M
M9M
M1N
M2N
M3N
M4N
M5N
M6N
M9N
M1P
M2P
M4P
M5P
M6P
M9P
M1R
M2R
M4R
M5R
M6R
M7R
M9R
M1S
M4S
M5S
M6S
M1T
M4T
M5T
M1V
M4V
M5V
M8V
M9V
M1W
M4W
M5W
M8W
M9W
M1X
M4X
M5X
M8X
M4Y
M7Y
M8Y
M8Z


In [9]:
print(toronto_venues.shape)
toronto_venues.head()

(2492, 7)


,PostalCode,PostalCode Latitude,PostalCode Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,M3A,43.752440,-79.329271,Brookbanks Park,43.751976,-79.332140,Park
1,M3A,43.752440,-79.329271,KFC,43.754387,-79.333021,Fast Food Restaurant
2,M3A,43.752440,-79.329271,Variety Store,43.751974,-79.333114,Food & Drink Shop
3,M4A,43.730421,-79.313320,Wigmore Park,43.731023,-79.310771,Park
4,M4A,43.730421,-79.313320,Memories of Africa,43.726602,-79.312427,Grocery Store


In [10]:
toronto_venues.groupby('PostalCode').count()

,PostalCode Latitude,PostalCode Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
PostalCode,,,,,,
M1B,2,2,2,2,2,2
M1C,3,3,3,3,3,3
M1E,3,3,3,3,3,3
M1G,4,4,4,4,4,4
M1H,2,2,2,2,2,2
M1J,4,4,4,4,4,4
M1K,5,5,5,5,5,5
M1L,9,9,9,9,9,9
M1M,10,10,10,10,10,10


In [11]:
# One hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# Add neighborhood column back to dataframe
toronto_onehot['PostalCode'] = toronto_venues['PostalCode']

# Move neighborhood column to the first column
nhood = toronto_onehot['PostalCode'] 
toronto_onehot.drop(labels=['PostalCode'], axis=1,inplace = True)
toronto_onehot.insert(0, 'PostalCode', nhood)

toronto_grouped = toronto_onehot.groupby('PostalCode').mean().reset_index()
toronto_grouped

,PostalCode,Afghan Restaurant,Airport,American Restaurant,Antique Shop,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,...,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,M1B,0.0,0.00,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,...,0.00,0.000000,0.000000,0.000000,0.000000,0.00000,0.0,0.000000,0.000000,0.000000
1,M1C,0.0,0.00,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,...,0.00,0.000000,0.000000,0.000000,0.000000,0.00000,0.0,0.000000,0.000000,0.000000
2,M1E,0.0,0.00,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,...,0.00,0.000000,0.000000,0.000000,0.000000,0.00000,0.0,0.000000,0.000000,0.000000
3,M1G,0.0,0.00,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,...,0.00,0.000000,0.000000,0.000000,0.000000,0.00000,0.0,0.000000,0.000000,0.000000
4,M1H,0.0,0.00,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,...,0.00,0.000000,0.000000,0.000000,0.000000,0.00000,0.0,0.000000,0.000000,0.000000
5,M1J,0.0,0.00,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,...,0.25,0.000000,0.000000,0.000000,0.000000,0.00000,0.0,0.000000,0.000000,0.000000
6,M1K,0.0,0.00,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,...,0.00,0.000000,0.000000,0.000000,0.000000,0.00000,0.0,0.000000,0.000000,0.000000
7,M1L,0.0,0.00,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,...,0.00,0.000000,0.000000,0.000000,0.000000,0.00000,0.0,0.000000,0.000000,0.000000
8,M1M,0.0,0.00,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,...,0.00,0.000000,0.000000,0.000000,0.000000,0.00000,0.0,0.100000,0.000000,0.000000
9,M1N,0.0,0.00,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,...,0.00,0.000000,0.000000,0.000000,0.000000,0.00000,0.0,0.000000,0.000000,0.000000


In [12]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [13]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['PostalCode']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['PostalCode'] = toronto_grouped['PostalCode']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,PostalCode,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Home Service,Pet Store,Dog Run,Fish Market,Fish & Chips Shop,Field,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant
1,M1C,Golf Course,History Museum,Bar,Yoga Studio,Falafel Restaurant,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space
2,M1E,Construction & Landscaping,Gym / Fitness Center,Park,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm
3,M1G,Korean Restaurant,Park,Coffee Shop,Business Service,Yoga Studio,Falafel Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Farmers Market
4,M1H,Playground,Trail,Yoga Studio,Falafel Restaurant,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Farm


## Cluster Neighborhoods
Run *k*-means to cluster the neighborhood into 5 clusters.

In [14]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('PostalCode', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([3, 3, 2, 0, 3, 3, 3, 3, 3, 0], dtype=int32)

Create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [15]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = toronto_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('PostalCode'), on='PostalCode')

# Where there are no venues a NaN will have been created, drop those rows and convert the cluster back to int
toronto_merged.dropna(inplace=True)
toronto_merged["Cluster Labels"]=toronto_merged["Cluster Labels"].astype(int)

toronto_merged.head() # check the last columns!

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,Parkwoods,43.752440,-79.329271,0,Food & Drink Shop,Park,Fast Food Restaurant,Yoga Studio,Falafel Restaurant,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space
1,M4A,North York,Victoria Village,43.730421,-79.313320,0,Park,French Restaurant,Intersection,Grocery Store,Yoga Studio,Falafel Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space
2,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.655120,-79.362640,3,Coffee Shop,Breakfast Spot,Gym / Fitness Center,Health Food Store,Theater,Spa,Event Space,Food Truck,Mexican Restaurant,Electronics Store
3,M6A,North York,"Lawrence Heights, Lawrence Manor",43.723125,-79.451589,3,Clothing Store,Dessert Shop,Cosmetics Shop,Toy / Game Store,Shopping Mall,Furniture / Home Store,Electronics Store,Rental Car Location,Chocolate Shop,Men's Store
4,M7A,Queen's Park,Queen's Park,43.661102,-79.391035,3,Coffee Shop,Sandwich Place,Café,Italian Restaurant,College Cafeteria,Indian Restaurant,Bubble Tea Shop,Pharmacy,Juice Bar,Bookstore


Visualise the clusters

In [16]:
# create map
map_clusters = folium.Map(location=[tor_lat, tor_long], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['PostalCode'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters